#code for
Fusion-based ensemble accuracy:  0.9473684210526315
Voting-based ensemble accuracy:  0.9473684210526315
Bagging-based ensemble accuracy:  0.956140350877193
Gradient Boosting accuracy:  0.956140350877193
Voting-based ensemble accuracy:  0.9824561403508771

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base estimators
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)

# 1. Fusion-based ensemble (averaging the predictions)
fusion_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2)], voting='soft')
fusion_clf.fit(X_train, y_train)
fusion_preds = fusion_clf.predict(X_test)
fusion_accuracy = accuracy_score(y_test, fusion_preds)

# 2. Voting-based ensemble (implemented by default in VotingClassifier)
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2)], voting='hard')
voting_clf.fit(X_train, y_train)
voting_preds = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, voting_preds)

# 3. Bagging-based ensemble
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_clf.fit(X_train, y_train)
bagging_preds = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, bagging_preds)

# 4. Gradient Boosting
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train, y_train)
gb_preds = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_preds)

# Neural tree ensembles are not directly implemented in popular libraries.

# Snapshot ensembles are not directly implemented in popular libraries.

# 7. Adversarial training is not applicable for this dataset and ensemble methods in sklearn.

# 8. Fast Geometric Ensembling (FGE) is not directly implemented in popular libraries.

# Soft Gradient Boosting is not directly implemented in popular libraries.

# Print accuracy scores
print("Fusion-based ensemble accuracy: ", fusion_accuracy)
print("Voting-based ensemble accuracy: ", voting_accuracy)
print("Bagging-based ensemble accuracy: ", bagging_accuracy)
print("Gradient Boosting accuracy: ", gb_accuracy)



Fusion-based ensemble accuracy:  0.9473684210526315
Voting-based ensemble accuracy:  0.9473684210526315
Bagging-based ensemble accuracy:  0.956140350877193
Gradient Boosting accuracy:  0.956140350877193


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create PyTorch DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define training and evaluation functions
def train(model, optimizer, criterion, train_loader, device):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            pred = output.argmax(dim=1)
            correct += (pred == y).sum().item()
    return correct / len(test_loader.dataset)

# Training parameters
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = 2
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Voting-based ensemble
models = [SimpleNN(input_dim, hidden_dim, output_dim).to(device) for _ in range(3)]
optimizers = [optim.Adam(model.parameters()) for model in models]
criterion = nn.CrossEntropyLoss()

for model, optimizer in zip(models, optimizers):
    for epoch in range(num_epochs):
        train(model, optimizer, criterion, train_loader, device)

# Evaluate the ensemble
ensemble_preds = []
for model in models:
    model_preds = []
    with torch.no_grad():
        for x in X_test:
            x = x.to(device)
            output = model(x)
            pred = output.argmax(dim=0)
            model_preds.append(pred.item())
    ensemble_preds.append(model_preds)

# Average the predictions of the ensemble
ensemble_preds = np.mean(ensemble_preds, axis=0)
ensemble_preds = np.round(ensemble_preds).astype(int)

# Calculate accuracy for the ensemble
ensemble_accuracy = accuracy_score(y_test.cpu().numpy(), ensemble_preds)

# Print accuracy score
print("Voting-based ensemble accuracy: ", ensemble_accuracy)


Voting-based ensemble accuracy:  0.9824561403508771
